In [20]:
import glob
import opencc

def make_alphabet(text_file_list,dest_file_path):
    res_dict = {}
    res_set = set()
    converter = opencc.OpenCC('s2t.json')
    for t_path  in text_file_list:
        with open(t_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                a = line.split(" ")[0]
                b = converter.convert(a)
                res_set.add(b.upper())
    conv_t2s = opencc.OpenCC('t2s.json')
    for t_c in res_set:
        s_c = conv_t2s.convert(t_c).lower()
        res_dict[t_c] = set([t_c, s_c])
    
    with open(dest_file_path, 'w+') as f:
        for key, ele in res_dict.items():
            _x = " ".join(list(ele))
            f.write(f"{_x}\n")

make_alphabet(glob.glob("ddata/TKHMTH2200/MTH1000/label_char/*.txt"),"ddata/quickviet/alphabet_MTH1000.txt")
make_alphabet(glob.glob("ddata/TKHMTH2200/MTH1200/label_char/*.txt"),"ddata/quickviet/alphabet_MTH1200.txt")
make_alphabet(glob.glob("ddata/TKHMTH2200/TKH/label_char/*.txt"),"ddata/quickviet/alphabet_TKH.txt")


In [ ]:
from osocr_tasks.athena.quickptg1 import prepare_pt
prepare_pt("/home/izhangxm/Desktop/work/main/VSDF/ddata/quickviet")








In [11]:
from neko_sdk.lmdb_wrappers.im_lmdb_wrapper import im_lmdb_wrapper
import glob
import os.path
import cv2
from pathlib import Path

def cal_bounding_box(ps):
    # 左下角，顺时针转圈, 返回左上角+右下角的信息
    x1,y1,x2,y2,x3,y3,x4,y4 =  ps
    _x1 = min(x1,x2,x3,x4)
    _y1 = min(y1,y2,y3,y4)
    _x2 = max(x1,x2,x3,x4)
    _y2 = max(y1,y2,y3,y4)
    return _x1,_y1,_x2,_y2

def quick_lmdb(mth_dataset_dir,dst,lang="None"):
    img_dir = Path(os.path.join(mth_dataset_dir,"img"))
    gt_dir = Path(os.path.join(mth_dataset_dir,"label_textline"))
    
    gt_file_path_list = glob.glob(str(gt_dir/"*.txt"))
    db=im_lmdb_wrapper(dst)

    for ii, char_file in enumerate(gt_file_path_list):
        char_file = Path(char_file)
        img_file_path = img_dir/ f"{char_file.stem}.png"
        
        if not img_file_path.exists():
            img_file_path = img_dir/ f"{char_file.stem}.jpg"
        
        if not img_file_path.exists():
            continue
            
        img_full = cv2.imread(str(img_file_path))
        if ii % 100 == 0:
            print(f"{ii+1}/{len(gt_file_path_list)} {char_file}")
        try:
            f = open(char_file, 'r')
            lines = f.readlines()
            for line in lines:
                eles = line.replace("\n", "").split(',')
                anno = eles[0]
                ps = [int(_) for _ in eles[1:]]
                x1,y1,x2,y2 = cal_bounding_box(ps)
                img = img_full[y1:y2, x1:x2,:]
                db.add_data_utf(img,anno,lang)
        except Exception as e:
            print(line)
            print(img_file_path)
        finally:
            f.close()
    db.end_this()

In [ ]:

mth_dataset_dir = "/home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/MTH1200"
dst = "/home/izhangxm/Desktop/work/main/VSDF/ddata/lmdb/MTH1200"
quick_lmdb(mth_dataset_dir, dst)

In [ ]:
mth_dataset_dir = "/home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/MTH1000"
dst = "/home/izhangxm/Desktop/work/main/VSDF/ddata/lmdb/MTH1000"
quick_lmdb(mth_dataset_dir, dst)


In [13]:
mth_dataset_dir = "/home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH"
dst = "/home/izhangxm/Desktop/work/main/VSDF/ddata/lmdb/TKH"
quick_lmdb(mth_dataset_dir, dst)


1/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_031_27_27.txt
101/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_028_26_22.txt
201/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_026_25_08.txt
301/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_013_26_02.txt
401/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_022_29_08.txt
501/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_034_26_17.txt
601/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_026_25_09.txt
701/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_011_27_20.txt
801/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_035_25_21.txt
901/999 /home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH/label_textline/0001_007_26

In [8]:
import cv2
img_path = "/home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/MTH1200/img/GL_8_5_10.jpg"
img_full = cv2.imread(img_path)

line = "四法世閒沙門婆羅門及長壽天心,5475,3308,5475,791,5682,791,5682,3308\n"
eles = line.replace("\n", "").split(',')
anno = eles[0]
ps = [int(_) for _ in eles[1:]]
x1,y1,x2,y2 = cal_bounding_box(ps)
img = img_full[y1:y2, x1:x2,:]
cv2.imwrite("a.jpg", img)


(3900, 6000, 3)

In [32]:
import lmdb

db_path = "ddata/MTH_DB/MTH1200"
env = lmdb.open(db_path)
with env.begin(write=False) as txn:
    length = txn.stat()['entries']
    print(length)
    keys = list(txn.cursor().iternext(values=False))
    print(len(keys))

env.close()

132454
132454


In [33]:
with open("aa.txt", 'w+') as f:
    for ele in keys:
        f.write(f"{ele.decode()}\n")
